In [1]:
import numpy as np 
import pandas as pd 
import math

In [3]:
print(f"Ghost_real : {len(df_G_Real)}, Unique ID : {len(df_G_Real['ID'].unique())},  Unique 10.000 : {len(df_G_Real.iloc[:5000,17].unique())}")
print(f"Time_real : {len(df_T_Real)}, Unique ID : {len(df_T_Real['ID'].unique())},  Unique 10.000 : {len(df_T_Real.iloc[:5000,10].unique())}")
print(f"Misc_real : {len(df_M_Real)}, Unique ID : {len(df_M_Real['ID'].unique())},  Unique 10.000 : {len(df_M_Real.iloc[:5000,21].unique())}")

Ghost_real : 926366, Unique MATCLE : 6590,  Unique 10.000 : 207
Time_real : 701441, Unique MATCLE : 14422,  Unique 10.000 : 436
Misc_real : 60228, Unique MATCLE : 42124,  Unique 10.000 : 2631


(48-57) : numbers

(65-90) : uppercase

(97-122) : lowercase 

# Full Pipeline for Fraud & Real Data

We will define a set of functions that perform data preparation in a Final Usable state for the classifier models.
We define these steps for each Fraud type Data : 
- Anonymization step of Batch Fraud data
- Formatting of Batch Fraud Data label (Fraud column values)
- Appending Batch Fraud Data to last update Clean Fraud Data
- Creating a 1/5 balanced dataset of Fraud data + Real data, based on DataFrames input


INPUT : 
- DataFrame of Fraud data type i (raw generated in csv format)
- DataFrame of Fraud data type i (anonymized full usable data)

## Anonymization Functions

In [2]:
## GHOST & TIMESHEET DATA

def encodePersonID_Gh_Time(df,L) :
    if len(df['PersonID'].unique())>len(L) :
        return 'Not enough IDs to assign to dataset'
    i=0
    j=0
    k=-1
    while i <len(df) : 
        val = df.iloc[i,0]
        k+=1
        print(k)
        while j<len(df) and val == df.iloc[j,0] : 
            df.iloc[j,0] = L.iloc[k,0]
            j+=1
        i=j
        # L.iloc[1:].reset_index(drop=True)
        # L.tail(-1).reset_index(drop=True, inplace = True)
        L.drop(index=L.index[0]).reset_index(drop=True)
        # print(f" i,j = {i,j} , val[i] = {val},  , L value =  {L.iloc[i,0]} , df id value =  {df.iloc[i,0]}" )
    return df,L.iloc[k:,]
    # return df,L 

In [16]:
## Misclass

def encodePersonID_Misc(df,L) : 
    for i in range(len(df)) :
        # print(f" i = {i} , L size =  {L.iloc[i,0]} , df size =  {df.iloc[i,0]}" )
        df.iloc[i,0] = L.iloc[i,0]
        # L.iloc[1:].reset_index(drop=True)
        # L.tail(-1).reset_index(drop=True, inplace = True)
        L.drop(index=L.index[0]).reset_index(drop=True)
        print(L.shape)
    return df,L.iloc[len(df):,]
    # return df,L

## Main Process Functions

In [4]:
# GHOST & TIMESHEET

def append_Last_Update_Fraud_GH_Time(df_fraud_new,df_Fraud_last_update,df_real_data,L,Fraud_Type) : 
    df_anonym, L_upd = encodePersonID_Gh_Time(df_fraud_new, L)

    # Modifications on Fraud data
    df_anonym.columns = df_anonym.columns.str.strip()
    if 'Explanation' in df_anonym.columns : 
        df_anonym.drop('Explanation', inplace=True, axis=1)
    if 'Explanation' in df_Fraud_last_update.columns : 
        df_Fraud_last_update.drop('Explanation', inplace=True, axis=1)

    
    # Labelization
    if Fraud_Type == "Ghost" : 
        df_anonym["Fraud"] = 1
        df_Fraud_last_update["Fraud"] = 1
    elif Fraud_Type == "Timesheet" : 
        df_anonym["Fraud"] = 2
        df_Fraud_last_update["Fraud"] = 2

    # Labelization real data
    if 'Fraud' not in df_real_data.columns : 
        df_real_data["Fraud"] = 0
    
    # Appending to latest update fraud data
    df_Fraud_upd = pd.concat([df_anonym,df_Fraud_last_update],ignore_index= True)
    
    # Appending to Real data
    n = df_Fraud_upd.shape[0]*5
    print(n)
    df_full = pd.concat([df_Fraud_upd,df_real_data.iloc[:n,:]],ignore_index= True)

    return L_upd, df_Fraud_upd, df_full
    

In [17]:
# Misclassif

def append_Last_Update_Fraud_Misc(df_fraud_new,df_Fraud_last_update,df_real_data,L) : 
    df_anonym, L_upd = encodePersonID_Misc(df_fraud_new, L)
    print(L.shape)
    
    # Treatments for json file
    df_anonym.columns = df_anonym.columns.str.strip()
    if 'Explanation' in df_anonym.columns : 
        df_anonym.drop('Explanation', inplace=True, axis=1)
    # df_anonym.drop('Position', inplace=True, axis=1)
    
    # Labelization
    df_anonym["Fraud"] = 3

    # Appending to latest update fraud data
    df_Fraud_upd_M = pd.concat([df_anonym,df_Fraud_last_update],ignore_index= True)
    
    # Appending to Real data
    n = df_Fraud_upd_M.shape[0]*5
    df_full_M = pd.concat([df_Fraud_upd_M,df_real_data.iloc[:n,:]],ignore_index= True)

    # Saving
    L_upd.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\L_upd_M2.csv",index=False)
    df_Fraud_upd_M.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_M_2.csv",index=False)
    df_full_M.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\df_full_M2.csv",index=False)
    
    return L_upd, df_Fraud_upd_M, df_full_M
    

# Pipeline Execution

## Ghost Fraud

### Load

In [108]:
file_name_g = "G2-5-6"
# file_name_g = "json_ghost"

In [109]:
## Load New Fraud Data
path_fr_g=r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Ghost\\"
df_fraud_new_g = pd.read_json(path_fr_g + file_name_g + ".json")

In [110]:
## Load real data
df_real_G = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_G_Real_anonym.csv")
## Load Last updates
df_fraud_last_upd_g = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_G_1.csv")
L_G = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\ArchiveBis\L_upd_G_1.csv")

### Process

In [ ]:
L_upd_G, df_Fraud_upd_G, df_full_G = append_Last_Update_Fraud_GH_Time(df_fraud_new_g, df_fraud_last_upd_g, df_real_G, L_G,"Ghost")

### Save updates

In [116]:
L_upd_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\L_upd_G_2.csv",index=False)
df_Fraud_upd_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_G_2.csv",index=False)
df_full_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\df_full_G_2.csv",index=False)    

## Timesheet Fraud

### Load

In [88]:
file_name_t = "T1-2-19-24-3-4-15-8-9-17-18"
# file_name_t = "T6&10_plus_models_testing_kpis_output"

In [89]:
## Load New Fraud Data
path_fr_t= r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Timesheet\\"
# df_fraud_new_t = pd.read_csv(path_fr_t+file_name_t+".csv")
df_fraud_new_t = pd.read_json(path_fr_t+file_name_t+".json")

In [92]:
## Load real data
df_real_T = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_T_Real_anonym.csv")
## Load last updates
df_fraud_last_upd_t = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_T_1.csv")
L_T = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\ArchiveBis\L_upd_T1.csv")

### Process

In [ ]:
L_upd_T, df_Fraud_upd_T, df_full_T = append_Last_Update_Fraud_GH_Time(df_fraud_new_t, df_fraud_last_upd_t, df_real_T, L_T, "Timesheet")

### Save updates

In [98]:
L_upd_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\ArchiveBis\L_upd_T2.csv", index=False)
df_Fraud_upd_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_T_2.csv",index=False)
df_full_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\Archive\df_full_T2.csv",index=False)    

## Misclassification Fraud

### Load

In [18]:
file_name_m = "M9--21"  # with no .json
# file_name_m = "M1-8"  # with no .json

In [19]:
## Load New Fraud Data
path_fr_m = r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Misclassification\\"
df_fraud_new_m = pd.read_json(path_fr_m+file_name_m+".json")

In [20]:
## Load real data
df_real_M = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_M_Real_anonym_uniformized.csv")

## Load last updates
df_fraud_last_upd_m = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_M_1.csv")
L_M = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\ID_Encoding\ArchiveBis\L_upd_M1.csv")

In [30]:
# L_M = L_M.iloc[1:].reset_index(drop=True)
# L_M = L_M.tail(-1)

### Process

In [ ]:
L_upd_M, df_Fraud_upd_M, df_full_M = append_Last_Update_Fraud_Misc(df_fraud_new_m, df_fraud_last_upd_m, df_real_M, L_M)

#### Updates are Autosaved for this case
---------------------------------------